# Online Laptop Store Inventory Analysis

In this project, we will utilise Python to build a way to answer different business questions about the online laptop store inventory. 

The key skills used in this project:
* analysing time and space complexity of algorithms
* preprocessing data to speed up algorithms
* sorting data and searching sorted data
* using efficient algorithms 

The data used has been adapted from the 'Laptop Prices' dataset which can be found on Kaggle. 

The goal of this project is to create a class which will represent the inventory of the online laptop store. The methods in this class will implement the queries that we want to answer about the inventory. The data will be preprocessed to make the queries run faster. 

Examples of queries we may wish to answer:
1. Given a laptop ID, find the corresponding data
2. Given the amount of money, find whether there are two laptops whose total price will equal the amount of money
3. Identify all laptops whose price falls within a given budget 

# Reading the inventory

Let's read in the file using a context manager, and display the header and first few rows of the dataset. 

In [1]:
import csv 
with open('laptops.csv') as file:
    laptop_data = list(csv.reader(file))
    
header = laptop_data[0]
rows = laptop_data[1:]

print(header)
print('\n')
print(rows[:4])

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537']]


# Creating an Inventory class
To create a class, we first need to implement a constructor. The construtor will take the name of the csv file as an argument, and read the rows that are contained within it. 

In [2]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        
inventory = Inventory('laptops.csv')

print(inventory.header)
print('\n')
print(len(inventory.rows))

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']


1303


# Finding a laptop based on a laptop ID 

In this project we will be making several changes to the Inventory class. To aid in keeping track of changes made, the Inventory class code will be copied each time and modified. 

The first thing we need to do to help us answer one of the queries on the inventory is a way to look up a laptop, given an ID. We will write a function which will take in the ID of the laptop as an argument, and return the full row of the laptop with that particular ID. 

In [3]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row 
        return None 

In [4]:
inventory = Inventory('laptops.csv')

print(inventory.get_laptop_from_id('3362737'))
print('\n')
print(inventory.get_laptop_from_id('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


None


# Improving ID lookups

Although the algorithm from the previous screen works, it requires us to look at every single row to find the one we are looking for. It has a higher time complexity. To reduce this, we can preprocess the data, and use a dictionary to check in constant time whether an ID exists, and then retrieve the relevant information. 

In [5]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row 
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row 
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.rows:
            if laptop_id in self.id_to_row:
                return self.id_to_row[laptop_id]
        return None 

In [6]:
inventory = Inventory('laptops.csv')

print(inventory.get_laptop_from_id_fast('3362737'))
print('\n')
print(inventory.get_laptop_from_id_fast('3362736'))

['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', 575]


None


# Comparing the Performance of the two algorithms

The get_laptop_from_id_fast() algorithm has a more optimised time complexity, as it uses more memory to store a dictionary and more time to create the instance. 

To compare the performance of the two algorithms, we can use the time module and compute a list of random ID's to test the algorithms and the time it takes to run. 

In [7]:
import time
import random 

N = 10000
ids = [str(random.randint(1000000, 9999999)) for _ in range(N)]


inventory = Inventory('laptops.csv')

total_time_no_dict = 0

for id in ids:
    start = time.time()
    inventory.get_laptop_from_id(id)
    end = time.time()
    total_time_no_dict += end - start
    
total_time_dict = 0

for id in ids:
    start = time.time()
    inventory.get_laptop_from_id_fast(id)
    end = time.time()
    total_time_dict += end - start
    
print("The total time taken for the algorithm with no dictionary is: ", total_time_no_dict)
print('\n')
print("The total time taken for the algorithm with a dictionary is: ", total_time_dict)


The total time taken for the algorithm with no dictionary is:  1.294353723526001


The total time taken for the algorithm with a dictionary is:  1.2109296321868896


We can see that the second algorithm is faster at locating the laptops given an ID. 

# Two Laptop Promotion - gift cards

Sometimes the online laptop store will offer a promotion including gift cards. The customer can only use the gift card once, and can only use it to buy up to 2 laptops. To make the best use out of the gift card, we want to ensure that each gift card can be used in full. This means, making sure there is a laptop worth exactly that amount, or two laptops that will add up to that exact amount. 

In [8]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row 
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row 
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.rows:
            if laptop_id in self.id_to_row:
                return self.id_to_row[laptop_id]
        return None 
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True 
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 

In [9]:
inventory = Inventory('laptops.csv')

print(inventory.check_promotion_dollars(1000))
print('\n')
print(inventory.check_promotion_dollars(442))

True


False


# Optimizing Laptop Promotion 

Once again, we can preprocess the data, and store the laptop prices in a set which will allow checking whether there is a laptop at a given price in constant time. 

In [10]:
class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row 
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row 
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.rows:
            if laptop_id in self.id_to_row:
                return self.id_to_row[laptop_id]
        return None 
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True 
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 
    
    def check_promotion_dollars_fast(self, dollars):
        for row in self.rows:
            if dollars in self.prices:
                return True 
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 

In [11]:
inventory = Inventory('laptops.csv')

print(inventory.check_promotion_dollars_fast(1000))
print('\n')
print(inventory.check_promotion_dollars_fast(442))

True


False


# Comparing Promotion Functions 

We can use similar syntax to before, to compare the performance of the two promotion functions. 

In [12]:
N = 100
prices = [random.randint(100, 5000) for _ in range(N)]

inventory = Inventory('laptops.csv')

total_time_no_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += end - start 
    
total_time_set = 0
for price in prices:
    start = time.time()
    inventory.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += end - start 
    
print("The total time taken for the algorithm with no set is: ", total_time_no_set)
print('\n')
print("The total time taken for the algorithm with a set is: ", total_time_set)

The total time taken for the algorithm with no set is:  2.3297410011291504


The total time taken for the algorithm with a set is:  2.371800422668457


# Finding Laptops Within a Budget 

To be able to write a query that finds all of the laptops that fall within the budget, we need to use binary search. If we sort the laptops by price, then the binary search will identify the first laptop in a sorted list which is larger than the budget. Therefore, the result will have all the laptops whose index is smaller than the index of the first laptop whose price is out of the budget. 

In [14]:
def row_price(row):
    return row[-1]

class Inventory():
    
    def __init__(self, csv_filename):
        with open(csv_filename) as file:
            data = list(csv.reader(file))
        self.header = data[0]
        self.rows = data[1:]
        for row in self.rows:
            row[-1] = int(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row 
        self.prices = set()
        for row in self.rows:
            self.prices.add(row[-1])
        self.rows_by_price = sorted(self.rows, key = row_price)
    
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            if row[0] == laptop_id:
                return row 
        return None 
    
    def get_laptop_from_id_fast(self, laptop_id):
        for row in self.rows:
            if laptop_id in self.id_to_row:
                return self.id_to_row[laptop_id]
        return None 
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            if row[-1] == dollars:
                return True 
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 
    
    def check_promotion_dollars_fast(self, dollars):
        for row in self.rows:
            if dollars in self.prices:
                return True 
        for row1 in self.rows:
            for row2 in self.rows:
                if row1[-1] + row2[-1] == dollars:
                    return True
        return False 
    
    def find_laptop_with_price(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price == target_price:                            
                return range_middle                        
            elif price < target_price:                           
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle - 1 
        price = self.rows_by_price[range_start][-1]
        if price != target_price:                  
            return -1                                      
        return range_start
    
    def find_first_laptop_more_expensive(self, target_price): 
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start


In [15]:
inventory = Inventory('laptops.csv')

print(inventory.find_first_laptop_more_expensive(1000))
print('\n')
print(inventory.find_first_laptop_more_expensive(10000))

683


-1
